In [1]:
import nltk, re, torch
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import goslate
import glob
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
snow = SnowballStemmer('english')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/niravjivani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niravjivani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def process_text(texts): 
    final_text_list=[]
    #gs = goslate.Goslate()    
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ''
        # translate any language to English
        # Define a delay between requests (in seconds)
        #request_delay = 1 
        
        #sent = gs.translate(sent, 'en')
        #print(sent)
        #time.sleep(request_delay)  # Introduce a delay between requests
        filtered_sentence=[]
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        for w in word_tokenize(sent):
            # Applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stopwords
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) # Final string of cleaned words
 
        final_text_list.append(final_string)
    return final_text_list

In [6]:
# Load the saved model and tokenizer
output_model_path = "./bert_paraphrase_model"
tokenizer = BertTokenizer.from_pretrained(output_model_path)
model = BertForSequenceClassification.from_pretrained(output_model_path)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

input_data = pd.DataFrame({
    'sentence1': ['Historical monuments and memorials endure as symbols of shared and collective memory and identity, embodying the stories, values, and hopes of societies across eras. Beyond safeguarding cultural heritage, these structures ignite introspection, dialogue, and contestation. Preserving their integrity amidst evolving societal, political, and environmental landscapes presents multifaceted challenges, necessitating nuanced approaches that balance conservation with inclusivity and contextual reinterpretation.  At the crossroads of memory, power, and heritage, these landmarks serve as arenas for negotiation, revealing the ongoing struggles for acknowledgment, representation, and reconciliation within society. Controversies surrounding certain monuments reveal the complexities web of memory politics, reflecting divergent viewpoints on history, selfhood, and dynamics power. By critically  examination of these dynamics , we gain insight into the memory’s constructions and the quest for historical veracity and justice in today’s discourse.'],
    'sentence2': ['Cricket is my favourite game of all other outdoor games in India. I used to play cricket in front of my house in the park with my school friends and neighbors. Cricket is a game that originated in British Empire (an English origin) however started playing in many countries. We need a bat and a ball to play this game. The cricket game came into vogue and gained popularity by the 18th century. There are two teams in cricket with 11 players in each, and two umpires to notice the faults according to the law and judge all the fair and unfair while playing the cricket. Before starting the game, a coin is tossed by the team’s captain to decide which team starts batting first and the other one bowling.Both of the teams get the chance to bat alternatively. However, the team that wins the toss does batting first, and the opposite team do bowling. Cricket has become one of the fascinating games in India from the observers’ point of view. When any national or international level cricket game is fixed to play, highly interested people become so excited a week before the start of the game. Many cricket lovers start booking tickets to see them live and on-site cricket games in the stadium instead of seeing them on TV or the news. Our country has become one of the most famous countries in cricket playing all over the world. Many times India has won the world cup and many test matches.'],
    'label': [2]  # Dummy labels for testing
})
input_data['sentence1'] = process_text(input_data['sentence1'])
input_data['sentence2'] = process_text(input_data['sentence2'])
# Prepare input tensors
inputs = tokenizer(
    input_data['sentence1'].tolist(), 
    input_data['sentence2'].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)
# Perform inference
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).cpu().numpy()

# Output the predictions
print("Predicted labels:", predictions)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Predicted labels: [0]
